In [108]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle

In [109]:
## Import and Convert images

data = []
labels = pd.read_csv("C:/Users/galan/Desktop/Stanford Classes/Quarter 9/Deep Learning/Project/train.csv")
labels["Id"] = labels["Id"].astype('category')
labels["Id_cat"] = labels["Id"].cat.codes                                              # Convert Id's to Numbers
id_label = labels[labels.columns[2]].tolist()                                          # Create a list with whale Id's
path = "C:/Users/galan/Desktop/Stanford Classes/Quarter 9/Deep Learning/Project/train" # Folder with Whale Images
img_size = 200                                                                         # Size that we will use to reshape images

def create_data():
    ii = 0
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)   # import each image and convert to grayscale
        new_array = cv2.resize(img_array, (img_size, img_size))                # convert images to desired size
        data.append([new_array, id_label[ii]])                                 # append images and Id labels to data list
        ii += 1 
        
create_data()
    

In [110]:
## Split data for Training and Test Sets

random.shuffle(data)   # Shuffle Data
percentage = 0.8       # Decide percentage of Training set

train_number = int(round(percentage*len(data)))
train_data = data[0:train_number]
test_data = data[train_number:]

In [111]:
## Create Training and Test Sets

X_train = []
Y_tr = []
X_test = []
Y_te = []

for features, label in train_data:
    X_train.append(features)
    Y_tr.append(label)

for features, label in test_data:
    X_test.append(features)
    Y_te.append(label)
    
X_train = np.array(X_train).reshape(-1,img_size,img_size,1)  # Convert to Numpy array for your NN
X_test = np.array(X_test).reshape(-1,img_size,img_size,1)    # Convert to Numpy array for your NN


In [113]:
## Make Label Sets Numpy Arrays of ones and zeros

Y_train = np.zeros((len(Y_tr), max(id_label)+1))
for ii in range(Y_train.shape[0]):
    Y_train[ii][Y_tr[ii]] = 1
    
Y_test = np.zeros((len(Y_te), max(id_label)+1))
for ii in range(Y_test.shape[0]):
    Y_test[ii][Y_te[ii]] = 1

## Convert Unique IDs to New Whale labels    
    
Y = np.concatenate((Y_train, Y_test), axis=0)
        
Suma = np.sum(Y, axis = 0)
for jj in range(len(Suma)):
    if Suma[jj] < 2:
        itemindex = np.where(Y[:,jj]==1)
        Y[itemindex,jj] = 0
        Y[itemindex,0] = 1

## Create the final version of Training and Test Label numpy arrays        
        
Y_train = Y[0:train_number, :]
Y_test = Y[train_number:, :]

In [143]:
## Save Sets into files to save time next time!

pickle_out = open("X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("Y_train.pickle","wb")
pickle.dump(Y_train, pickle_out)
pickle_out.close()

pickle_out = open("X_test.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("Y_test.pickle","wb")
pickle.dump(Y_test, pickle_out)
pickle_out.close()

pickle_out = open("labels.pickle","wb")
pickle.dump(labels, pickle_out)
pickle_out.close()